# Практическая работа 1
## Тема: Загрузка данных
## Цель работы: 
Изучение технологии извлечения данных из различных источников: CSV-файлы, Excel-файлы, JSON-файлы, HTML-файлы, SQL-базы данных.

### Часть 1. Чтение CSV-файла

С использованием функции pd.read_csv() выполните следующие операции по чтению данных для датасета titanic.csv:

1. прочитайте файл и переведите его в датафрейм, в качестве индекса используйте столбец 'PassengerId';

In [ ]:
import pandas as pd

pd.read_csv("titanic.csv", index_col="PassengerId")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


2. извлеките из файла и сохраните в датафрейм первые 10 строк столбцов 'Name', 'Sex' и 'Survived';

In [6]:
pd.read_csv("titanic.csv", nrows=10, usecols=["Name", "Sex", "Survived"])

,Survived,Name,Sex
0,0,"Braund, Mr. Owen Harris",male
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female
2,1,"Heikkinen, Miss. Laina",female
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female
4,0,"Allen, Mr. William Henry",male
5,0,"Moran, Mr. James",male
6,0,"McCarthy, Mr. Timothy J",male
7,0,"Palsson, Master. Gosta Leonard",male
8,1,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female
9,1,"Nasser, Mrs. Nicholas (Adele Achem)",female


3. прочитайте файл и задайте новый набор имен для столбцов в нижнем регистре;

In [42]:
names = "survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked".split(",")
pd.read_csv("titanic.csv", header=0, names=names)

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


4. прочитайте файл, пропустив первые 100 строк, а затем считайте следующие 8 строк;

In [43]:
pd.read_csv("titanic.csv", skiprows=100, nrows=8, names=names)

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
100,0,2,"Kantor, Mr. Sinai",male,34.0,1,0,244367,26.0000,NaN,S
101,0,3,"Petranec, Miss. Matilda",female,28.0,0,0,349245,7.8958,NaN,S
102,0,3,"Petroff, Mr. Pastcho (""Pentcho"")",male,NaN,0,0,349215,7.8958,NaN,S
103,0,1,"White, Mr. Richard Frasar",male,21.0,0,1,35281,77.2875,D26,S
104,0,3,"Johansson, Mr. Gustaf Joel",male,33.0,0,0,7540,8.6542,NaN,S
105,0,3,"Gustafsson, Mr. Anders Vilhelm",male,37.0,2,0,3101276,7.9250,NaN,S
106,0,3,"Mionoff, Mr. Stoytcho",male,28.0,0,0,349207,7.8958,NaN,S
107,1,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,NaN,S


5. прочитайте первые 10 строк из файла, пропустив строки 1, 3, 7.

In [24]:
pd.read_csv("titanic.csv", skiprows=[1, 3, 7], nrows=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
3,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
4,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
5,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
6,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
7,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
8,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
9,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S


### Часть 2. Чтение НТМL-файлов из Интернета

Изучив материал лекции, приведите свой пример на запись НТМL-таблицы в один или несколько объектов DataFгame. Продемонстрируйте возможность извлечения части таблицы.

In [25]:
import pandas as pd
import webbrowser as web
from os import path


url = "https://www.cnews.ru/reviews/" \
"dbaas2023/review_table/5c3bf5dc1f2f4058c7bcc51019984c37c1acf3f3"

table, *_ = pd.read_html(url)
table = table.set_index("Место")

raiting = table.iloc[:5, :3]

raiting.to_html("rating.html")
path = path.abspath("rating.html")
web.open(f"file://{path}")


True

### Часть 3. Чтение и запись JSОN-файлов

Изучив материал лекции, приведите свой пример работы с JSОN-файлом.

In [26]:
import pandas as pd

rating, *_ = pd.read_html("rating.html")

raiting.to_json("rating.json", force_ascii=False, indent=4)
raiting = pd.read_json("rating.json")

raiting

,Провайдер,Сервис,Сумма баллов
1,Selectel,Облачные базы данных (Managed Databases),643
2,Yandex Cloud,Managed Service,639
3,VK Cloud,Cloud Databases,606
4,---,Облачные базы данных,541
5,Cloud,Relational Database Service,519


## Часть 4. Работа с базой данных SQL
1. Используя встроенную Python-библиотеку для работы с базами данных - sqlite3 создайте базу данных для хранения сведений о работе кафе со следующей структурой:

    ![Схема бд](schema.png)

    Самостоятельно подумайте, какие типы данных следует выбрать для полей таблиц.

2. Добавьте в таблицу "Блюда" 5 любых блюд, в таблицу "Заказы" - 3 заказа и в каждом заказе по 2-3 блюда (таблица "Заказано").

In [ ]:
import sqlite3 as sl
from os import path
from subprocess import run
import pandas as pd

# 1 и 2
if not path.exists("cafe.db"):
    with open("init.sql", encoding="UTF-8") as f:
        run(["python", "-m", "sqlite3", "cafe.db"], stdin=f)

3. С помощью метода read_sql выполните извлечение данных из базы данных и представьте в виде датафрейма:

- информацию о заказанных блюдах со следующими столбцами: номер заказа, название блюда, цена.

In [ ]:
import sqlite3 as sl


with sl.connect("cafe.db") as conn:

    query = """
        select o.id as order_id, d.name as dish_name, d.price as dish_price
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

items


,order_id,dish_name,dish_price
0,1,Пицца Маргарита,12.99
1,1,Салат Цезарь,8.50
2,1,Томатный суп,6.75
3,2,Томатный суп,6.75
4,2,Спагетти Карбонара,10.25
5,3,Спагетти Карбонара,10.25
6,3,Тирамису,7.00
7,1,Пицца Маргарита,12.99
8,1,Спагетти Карбонара,10.25
9,2,Салат Цезарь,8.50


- сумму к оплате для каждого заказа.

In [ ]:
import sqlite3 as sl


with sl.connect("cafe.db") as conn:

    query = """
        select o.id as order_id, sum(d.price) as total
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
        group by o.id
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

items

,order_id,total
0,1,97.96
1,2,63.50
2,3,68.25


- названия всех блюд, у которых цена меньше 300 руб.

In [ ]:
import sqlite3 as sl


with sl.connect("cafe.db") as conn:

    query = "select name from dishes where price < 300"

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

items

,name
0,Пицца Маргарита
1,Салат Цезарь
2,Томатный суп
3,Спагетти Карбонара
4,Тирамису
5,Бургер
6,Суши
7,Салат Оливье
8,Ризотто с морепродуктами
9,Шоколадный торт


- все блюда заказа с номером 2 (или другим номером).

In [ ]:
import sqlite3 as sl


with sl.connect("cafe.db") as conn:

    # 3.4
    query = """
        select d.*
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
        where o.id = 2
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

items

,id,name,price
0,3,Томатный суп,6.75
1,4,Спагетти Карбонара,10.25
2,2,Салат Цезарь,8.50
3,5,Тирамису,7.00
4,2,Салат Цезарь,8.50
5,5,Тирамису,7.00
6,2,Салат Цезарь,8.50
7,5,Тирамису,7.00


- список блюд по убыванию цены.

In [ ]:
import sqlite3 as sl


with sl.connect("cafe.db") as conn:

    # 3.5
    query = "select * from Dishes order by price desc"

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

items

,id,name,price
0,7,Суши,20.00
1,12,Суши,20.00
2,17,Суши,20.00
3,9,Ризотто с морепродуктами,18.25
4,14,Ризотто с морепродуктами,18.25
5,19,Ризотто с морепродуктами,18.25
6,6,Бургер,15.50
7,11,Бургер,15.50
8,16,Бургер,15.50
9,1,Пицца Маргарита,12.99


Измените расчёт суммы к оплате, предусмотрев скидку 5% при наличии в заказе 3 и более блюд.

In [ ]:
import sqlite3 as sl


with sl.connect("cafe.db") as conn:


    # 3.6
    query = """
        select o.id as order_id, count(d.name) as count, sum(d.price) as total
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
        group by o.id
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)
    mask = items["count"] >= 3
    items.loc[mask, "total"] = (items.loc[mask, "total"] * (1 - 0.05)).round(2)

items

,order_id,count,total
0,1,9,93.06
1,2,8,60.32
2,3,8,64.84


4. Создайте датафреймы для хранения информации о трёх новых блюдах, двух новых заказах, в каждом из которых по 2-3 блюда. С помощью метода to_sql выполните запись в базу данных.

In [18]:
import sqlite3 as sl
import pandas as pd


with sl.connect("cafe.db") as conn:

    # 4
    dishes_data = {
        "name": ["Бургер", "Суши", "Салат Оливье", "Ризотто с морепродуктами", "Шоколадный торт"],
        "price": [15.50, 20.00, 7.80, 18.25, 6.50]
    }
    dishes_df = pd.DataFrame(dishes_data)

    orders_data = {
        "date": ["2024-01-15", "2024-02-17", "2024-03-10"]
    }
    orders_df = pd.DataFrame(orders_data)

    order_items_data = {
        "order_id": [1, 1, 2, 2, 3, 3],
        "dish_id": [1, 4, 2, 5, 3, 4]
    }
    order_items_df = pd.DataFrame(order_items_data)

    dishes_df.to_sql("Dishes", conn, if_exists="append", index=False)
    orders_df.to_sql("Orders", conn, if_exists="append", index=False)
    order_items_df.to_sql("OrderItems", conn, if_exists="append", index=False)

#### Экспорт в html

In [1]:
from os import system

system("jupyter nbconvert --to html practice_1.ipynb")

[NbConvertApp] Converting notebook practice_1.ipynb to html
[NbConvertApp] Writing 337618 bytes to practice_1.html


0